In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

import evosax

from typing import NamedTuple
from functools import partial
import math


In [3]:
import json
import drawsvg

In [10]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.receptance as r
import xylo.sweep as xs

import xylo.loss.receptance as xlr

import xylo.database as db
import xylo.just

import xylo.find_lengths as find_lengths


In [44]:
layout = xylo.just.diamond_11_partch # ._replace(reference_hz = 784*2)
freqs = layout.get_freqs()
freqs = [freqs[0], freqs[1], freqs[-2], freqs[-1]]
d = 0.010
# d = 0.006
bar_template = t.BarProps(width = 0.032, depth = d, length = 100, elements = 1, min_depth = d)

# mat = xylo.woods.aluminium_6060T5
mat = xylo.woods.steel_3679_1_300


sum_lengths = 0
count = 0
lengths = {}

# on test bar 200 F#5, the drilling and finishing reduced the frequency from 745 to 735. so, make the cut at 1.5% above the target frequency
overshoot = 1 # 1.015

for i, freq in enumerate(freqs):
  name = i
  l, freq_est = find_lengths.sweep_find_length(mat, freq * overshoot, bar_template)
  print(f"*** note {name}: ***")
  print(f"    target freq {freq}, length {l}, est cut freq {freq_est}")
  sum_lengths += l
  count += 1
  lengths[i] = l

print(f"total length {sum_lengths}, count {count}")


*** note 0: ***
    target freq 2156.0, length 0.1539306640625, est cut freq 2153.6236312776036
*** note 1: ***
    target freq 1764.0, length 0.17041015625, est cut freq 1764.7776227037773
*** note 2: ***
    target freq 356.3636363636364, length 0.380859375, est cut freq 356.79752428003974
*** note 3: ***
    target freq 285.0909090909091, length 0.42578125, est cut freq 285.6242218609458
total length 1.1309814453125, count 4


In [20]:
print(lengths)

{0: 0.11875000000000004, 1: 0.13144531250000002, 2: 0.13300781250000004, 3: 0.14902343750000002, 4: 0.14707031250000002, 5: 0.14570312500000004, 6: 0.1611328125, 7: 0.166796875, 8: 0.1611328125, 9: 0.157421875, 10: 0.17656250000000004, 11: 0.18027343750000002, 12: 0.18281250000000002, 13: 0.1740234375, 14: 0.1787109375, 15: 0.19765625, 16: 0.19765625, 17: 0.19765625, 18: 0.19765625, 19: 0.19765625, 20: 0.19765625, 21: 0.22109375000000006, 22: 0.21640625000000005, 23: 0.21328125000000006, 24: 0.22421875000000005, 25: 0.21835937500000008, 26: 0.24218750000000008, 27: 0.23398437500000008, 28: 0.24218750000000008, 29: 0.24765625000000005, 30: 0.2617187500000001, 31: 0.265234375, 32: 0.2679687500000001, 33: 0.29687500000000006, 34: 0.2929687500000001, 35: 0.32812500000000006}


In [45]:
def mklens(lens, filename, offsetx = 1.3):
  d = drawsvg.Drawing('297mm', '210mm')
  d.append(drawsvg.Rectangle(0, 0, '297mm', '210mm', stroke='green', stroke_width=2, fill='none'))
  d.append(drawsvg.Text(filename, '5mm', '20mm', '8mm'))
  grp = drawsvg.Group()
  y = 10
  for k,v in lens.items():
    mm = v * 1000 + offsetx
    # print(k,v)
    grp.append(drawsvg.Text(f'{k}', '2mm', f'{mm-1}mm', f'{y + 3}mm'))
    grp.append(drawsvg.Circle(f'{mm}mm', f'{y + 1}mm', '0.5mm'))
    grp.append(drawsvg.Rectangle(f'{offsetx}mm', f'{y}mm', f'{mm}mm', f'0.5mm', stroke='red', stroke_width=1))
  grp.append(drawsvg.Circle(f'{offsetx}mm', f'{y+1}mm', '1mm'))

  d.append(grp)

  d.save_html(f'data/layout/{filename}.html')
  return d

mklens(lengths, 'glock-lengths')
